# Convert existing Rmarkdown file to DSC executables

This tutorial demonstrates a convenient way to construct DSC based on Rmarkdown files being used in Rstudio interactive sessions. Material used can be found in [DSC2 vignettes repository](https://github.com/stephenslab/dsc2/tree/master/vignettes/mash).

Caution that running DSC with source code provided from Rmarkdown files is meant ease DSC development, but is more error prone because users may make more variable or code chunk matching mistakes between DSC interface and the Rmarkdown file. This feature is intended to prototype a benchmark. Users are not recommanded to use Rmd files in the finalized benchmark.

The Rmarkdown file is part of the [`mashr` package intro vigenette](https://stephenslab.github.io/mashr/docs/intro_mash.html). Here we run in DSC the steps of simulating data, compute covariance matrices (prior), and fit the `mash` model. The DSC file is specified as:

```
#!/usr/bin/env dsc

simulate: R(library(mashr)) + simulate_*@intro_mash.Rmd
  n_effects: 500
  n_cond: 5
  $data: data

get_cov: R(library(mashr)) + cov*@intro_mash.Rmd
  data: $data
  $U_c: U.c 

fit: R(library(mashr)) + fit*@intro_mash.Rmd
  U_c: $U_c
  data: $data
  $m_c: m.c
  @ALIAS: U.c = U_c
 
DSC:
  run: simulate * get_cov * fit
  R_libs: mashr@stephenslab/mashr (0.2.5+)
  output: mash_result
```

Take `simulate_*@intro_mash.Rmd` for example: it means the module executable comes from code chunks matching `simulate_*` pattern in file `intro_mash.Rmd`. The code chunk matching pattern follows from [UNIX wildcard](http://www.robelle.com/smugbook/wildcard.html) convention. When only the Rmarkdown filename (the part after `@`) is specified, all code chunks in the Rmarkdown file will be loaded.

The extracted code along with this DSC script can be found in DSC code browswer [`mash_result.html`](mash_result.html). To run the DSC:

In [1]:
%cd ~/GIT/dsc2/vignettes/mash

/home/gaow/GIT/dsc2/vignettes/mash

In [2]:
! ./settings.dsc

INFO: Checking R library mashr@stephenslab/mashr ...
INFO: Checking R library dscrutils@stephenslab/dsc2/dscrutils ...
INFO: DSC script exported to mash_result.html
INFO: Constructing DSC from ./settings.dsc ...
INFO: Building execution graph & running DSC ...
DSC: 100%|████████████████████████████████████████| 4/4 [00:08<00:00,  1.92s/it]
INFO: Building DSC database ...
INFO: DSC complete!
INFO: Elapsed time 13.973 seconds.


Note that warning messages are displayed when module executables are loaded from Rmarkdown files, to remind users to finalize the DSC with formal scripts rather than some dynamic document.